In [1]:
%%bash
wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
unzip trainingandtestdata.zip

Archive:  trainingandtestdata.zip


URL transformed to HTTPS due to an HSTS policy
--2018-09-30 16:24:58--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
cs.stanford.edu (cs.stanford.edu) をDNSに問いあわせています... 171.64.64.64
cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 81363704 (78M) [application/zip]
`trainingandtestdata.zip.4' に保存中

     0K .......... .......... .......... .......... ..........  0%  159K 8m20s
    50K .......... .......... .......... .......... ..........  0% 2.22M 4m27s
   100K .......... .......... .......... .......... ..........  0%  371K 4m9s
   150K .......... .......... .......... .......... ..........  0%  352K 4m3s
   200K .......... .......... .......... .......... ..........  0%  334K 4m2s
   250K .......... .......... .......... .......... ..........  0% 12.4M 3m22s
   300K .......... .......... .......... .......... ..........  0% 8.37M 2m55s
   350K .......... .......... .......... .......... ..........  0% 

In [2]:
#reference 
# http://sucrose.hatenablog.com/entry/2013/10/29/001041
# http://help.sentiment140.com/for-students/

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import codecs
import seaborn as sns
from sklearn import svm
%matplotlib inline

with codecs.open("training.1600000.processed.noemoticon.csv", "r", "UTF-8", "ignore") as file:
    df = pd.read_table(file, delimiter=",", header=None)
    print(df)

         0           1                             2         3  \
0        0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1        0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2        0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3        0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4        0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
5        0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY   
6        0  1467811592  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
7        0  1467811594  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
8        0  1467811795  Mon Apr 06 22:20:05 PDT 2009  NO_QUERY   
9        0  1467812025  Mon Apr 06 22:20:09 PDT 2009  NO_QUERY   
10       0  1467812416  Mon Apr 06 22:20:16 PDT 2009  NO_QUERY   
11       0  1467812579  Mon Apr 06 22:20:17 PDT 2009  NO_QUERY   
12       0  1467812723  Mon Apr 06 22:20:19 PDT 2009  NO_QUERY   
13       0  1467812771  Mon Apr 06 22:20:19 PDT 2009  NO_QUERY   
14       0

In [3]:
df.head()
tmp_table = df.head(10)
tmp_table = df

In [4]:
num=100
pos = df[tmp_table[0] == 4].head(num)
neg = df[tmp_table[0] == 0].head(num)

In [5]:
tmp_table = pd.concat([pos, neg])

In [6]:
label = tmp_table[0].values

In [7]:
from gensim import corpora, models, similarities

In [8]:
documents = []

for i in tmp_table.index:
    documents.append(tmp_table.loc[i][5])

In [9]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist and word[0] != '@']
         for document in documents]

In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict') # store the dictionary, for future reference

In [11]:
print(len(dictionary))

1233


In [12]:
corpus = [dictionary.doc2bow(text) for text in texts]
tmp_corpus = corpus
corpus = []

for tmp in tmp_corpus:
    ids = [0] * len(dictionary)
    for id_of_word , cnt in tmp:
        ids[id_of_word] = ids[id_of_word] + cnt
    corpus.append(ids)

In [13]:
dictionary.save_as_text('/tmp/deerwester.dict.txt')

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(corpus, label)

GaussianNB(priors=None)

In [48]:
documents = [
"The antidote to government by a powerful few is government by the organized, energized many. This National Voter Registration Day, make sure you're registered, vote early if you can, or show up on November 6. This moment is too important to sit out.",
"Thank you to Chairman Kim Jong Un for keeping your word & starting the process of sending home the remains of our great and beloved missing fallen! I am not at all surprised that you took this kind action. Also, thank you for your nice letter - l look forward to seeing you soon!",
"North Korean Leader Kim Jong Un just stated that the “Nuclear Button is on his desk at all times.” Will someone from his depleted and food starved regime please inform him that I too have a Nuclear Button, but it is a much bigger & more powerful one than his, and my Button works!",
"We will always remember everyone we lost on 9/11, thank the first responders who keep us safe, and honor all who defend our country and the ideals that bind us together. There's nothing our resilience and resolve can’t overcome, and no act of terror can ever change who we are.",
"Thank you West Virginia - I love you!",
]

texts = [[word for word in document.lower().split() if word not in stoplist and word[0] != '@']
         for document in documents]

text_corpus =  [dictionary.doc2bow(text) for text in texts]

In [49]:
def convert_to_vector(tmp_corpus, dictionary):
    ret = [0] * len(dictionary)
    print(tmp_corpus)
    for id_of_word , cnt in tmp_corpus:
        ret[id_of_word] = ret[id_of_word] + cnt
    return ret

In [50]:
converted = [convert_to_vector(text, dictionary) for text in text_corpus]

[(16, 1), (28, 1), (34, 1), (44, 1), (51, 1), (58, 1), (78, 2), (102, 2), (152, 1), (157, 1), (187, 1), (262, 2), (276, 1), (591, 1), (609, 1), (895, 1), (984, 1)]
[(2, 1), (6, 1), (21, 1), (34, 4), (53, 2), (59, 1), (78, 1), (80, 1), (82, 1), (245, 1), (344, 1), (392, 1), (497, 1), (697, 1), (1040, 1), (1061, 1)]
[(2, 1), (12, 1), (13, 1), (21, 1), (40, 1), (43, 1), (45, 1), (51, 1), (58, 1), (59, 2), (63, 1), (64, 1), (102, 2), (144, 1), (189, 1), (207, 1), (236, 1), (250, 1), (344, 1), (570, 1), (720, 2), (1072, 1)]
[(37, 1), (55, 1), (58, 1), (59, 1), (269, 1), (314, 3), (344, 1), (392, 2), (536, 2), (570, 1), (753, 1), (799, 1), (910, 3), (1046, 1)]
[(2, 1), (3, 1), (6, 1), (34, 1), (355, 1)]


In [51]:
gnb.predict(converted)

array([4, 4, 0, 0, 4])